In [1]:
!sudo pip install -q transformers --upgrade

In [ ]:
!pip install autoawq

     |████████████████████████████████| 20.4MB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 184kB 121.7MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 112.6MB/s eta 0:00:01


In [4]:
import torch

In [1]:
!pip freeze

absl-py==1.4.0
accelerate==0.21.0
aiohttp==3.8.5
aiosignal==1.3.1
alembic==1.11.1
annotated-types==0.6.0
anyio==3.7.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.2.1
async-generator==1.10
async-timeout==4.0.2
attributedict==0.3.0
attrs==23.1.0
autoawq==0.1.7
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.12.2
bitsandbytes==0.41.1
bleach==4.1.0
blessings==1.7
boltons==23.0.0
bqplot==0.12.39
brotlipy==0.7.0
cachetools==5.3.1
certifi==2023.5.7
certipy==0.1.3
cffi==1.15.1
chardet==5.2.0
charset-normalizer==2.0.4
click==8.1.7
cloudpickle==1.6.0
cmake==3.27.1
codecov==2.1.13
colorama==0.4.6
coloredlogs==15.0.1
colour-runner==0.1.1
conda==23.5.0
conda-package-handling==2.1.0
conda-package-streaming==0.8.0
configparser==6.0.0
coverage==7.3.2
cryptography==39.0.1
DataProperty==1.0.1
datasets==2.14.4
decorator==5.1.1
deepdiff==6.7.1
defusedxml==0.7.1
dill==0.3.7
distlib==0.3.7
distro==1.8.0
docker-pycreds==0.4.0
einops==0.6.1
entrypoints==0.4
evaluate==0.4.0
exc

In [5]:
new_model_name = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained(model)

In [7]:
# Load LLaMA tokenizer
from transformers import AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [10]:
from autoawq import AWQQuantizer

quantizer = AWQQuantizer(
    model=model,
    tokenizer=tokenizer,
    bnb_config=bnb_config,
    quant_method="awq",
    num_epochs=10,
    lr=1e-4,
    batch_size=1,
    eval_batch_size=1,
    max_length=2048,
    device="cuda",
)

ModuleNotFoundError: No module named 'autoawq'

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


In [ ]:
quantizer.quantize(
    input_data=dataset["train"],
    valid_data=dataset["validation"],
)